# AIGC在药物设计中的应用：激酶相似性分析
## 演示激酶的相似性分析，包括序列相似性、口袋相似性（KiSSim指纹）、相互作用指纹相似性、配体谱相似性等多个视角。


## 引言

激酶是一类重要的酶，参与细胞内信号转导、细胞周期调控等关键生物过程。在药物设计中，激酶作为靶点的研究尤为重要，尤其是在癌症等疾病的治疗中。理解激酶之间的相似性有助于发现新药物、优化现有药物以及预测药物的潜在靶点。

本笔记本将介绍如何从多个视角分析激酶的相似性，包括序列相似性、口袋相似性、相互作用指纹相似性和配体谱相似性，并通过综合分析提升对激酶功能和药物作用机制的理解。

---

## 导入必要的库

首先，我们需要导入所需的Python库。这些库包括数据处理、生物信息学分析、机器学习模型以及可视化工具。


In [6]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, rdMolDescriptors

from Bio import SeqIO
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# For binding pocket identification
!pip install biopython pymol  # Uncomment if not already installed

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# 防止中文乱码
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False



ERROR: Could not find a version that satisfies the requirement pymol (from versions: none)
ERROR: No matching distribution found for pymol


In [7]:
from pathlib import Path
import os

# 获取当前工作目录
HERE = Path(os.getcwd())
DATA = HERE / 'data'
if not DATA.exists():
    DATA.mkdir(parents=True, exist_ok=True)
print(DATA)

/Users/wangyang/Desktop/AIGC-in-drug-design/03_kinase/02_Kinase sequence similarity/data


## 数据加载与预处理

### 加载激酶序列数据

假设我们有一个包含激酶序列的FASTA文件`kinases.fasta`，我们将加载并解析这些序列。

In [13]:
# 定义数据路径
sequence_data_path = DATA / 'kinases.fasta'

# 解析FASTA文件
kinase_sequences = {}
for record in SeqIO.parse(sequence_data_path, "fasta"):
    kinase_id = record.id
    sequence = str(record.seq)
    kinase_sequences[kinase_id] = sequence
    
# 加载进度条，这里用tqdm库实现
from tqdm import tqdm
# 加载进度条
list(tqdm(kinase_sequences))
print(f"加载了 {len(kinase_sequences)} 个激酶序列。")



100%|██████████| 825653/825653 [00:00<00:00, 5675058.18it/s]

加载了 825653 个激酶序列。


### 加载激酶结构数据

假设我们有一个包含激酶PDB文件的目录`structures/`，我们将加载这些结构以进行口袋识别。

In [19]:
# 定义结构数据路径
structure_dir = DATA /'structures'

# 列出所有PDB文件
pdb_files = [f for f in os.listdir(structure_dir) if f.endswith('.pdb')]

print(f"加载了 {len(pdb_files)} 个激酶结构文件。")


加载了 16 个激酶结构文件。
